#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 04 - Part 03 - Bash Variables and Pipelines</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

* Variables are assigned with `=` without spaces around the operator
* All variables essentially are treated as if they stored string information
* `$` must precede variables so that bash knows these aren't meant to be interpreted as text
* Variables that don't exist produce empty strings
* `{}` are used to ensure there's no ambiguity and perform string operations

```sh
$ msg="Hello, variable"
$ echo msg
msg
$ echo $msg
Hello, variable
$ echo ${msg}
Hello, variable
$ echo ${msg:3:8}  # take 8 characters starting from position 3
lo, vari
```

* NO SPACE AROUND `=`, very important
* `msg = "Hello, variable"` interpreted by bash as running the program `msg` with `=` and `"Hello, variable"` passed as arguments
* `msg= "Hello, variable"` interpreted as running the program `"Hello, variable"` with the environment variable `msg` set to the empty string (more on environment variables later)
* This and other quirks are the result of bash's lineage and its focus as an interpreted, interactive language

```sh
$ msg = "Hello, variable"
bash: msg: command not found
$ msg= "Hello, variable"
bash: Hello, variable: command not found
```

* Since variables store text doing math is a little harder, modern versions of bash help us though

```sh
x=1
y=2
echo $x + $y  # prints "1 + 2"
echo "$x + $y"  # prints "1 + 2", double quotes allow expansion
echo '$x + $y'  # prints "$x + $y", single quotes prevent expansion
echo $((x+y))  # prints "3"
((x+=1))  # increment x by 1
echo $((x+y))  # prints "4"
```

* Processes can be run in `$()` constructs and evaluate to whatever was written to stdout
* Like other languages `;` can be used to chain commands together as a single unit

```sh
$ pwd
/home/localek10/workspace/BMEISWorkshops/13_bash_linux
$ parent_dir=$(cd ..; pwd)  # move to the parent directory and print current path
$ echo $parent_dir
/home/localek10/workspace/BMEISWorkshops
```

* Arguments to commands are tokenized by spaces, whether they come from variables or not
* Double-quotes can be used to tie arguments together, single-quotes to pass the value literally without variable expansion
* `\` can be used to escape characters and not interpret them for special meaning

```sh
$ x="hello, argument"
$ python -c 'import sys; print(sys.argv)' $x
['-c', 'hello,', 'argument']
$ python -c 'import sys; print(sys.argv)' "$x"
['-c', 'hello, argument']
$ python -c 'import sys; print(sys.argv)' '$x'
['-c', '$x']
$ python -c 'import sys; print(sys.argv)' hello,\ argument
['-c', 'hello, argument']
```

* Filenames can be selected using a simple regular expression system called filename expansion when assigning to variables or used as arguments
* Uses `*` to stand for anything, `?` for one character, `{}` to present a list of choices

```sh
$ echo /bin/w*
/bin/wdctl /bin/which /bin/whiptail
$ echo /bin/??d
/bin/pwd /bin/red /bin/sed
```

* If a pattern matches no existing files it stay unexpanded:

```sh
$ echo foo*bar
foo*bar
```

* All running programs have a process identifier or PID number which is unique, `$$` stores current shell's PID
* Programs return an exist status integer, 0 means success, positive values indicate various failure results, negative numbers indicate errors
* Exit status captured in variable `$?`, used for decisions in conditional blocks:

```sh
$ echo "Success!"
Success!
$ echo $?
0
```

* Appending `&` to a command causes it to run in the background without control of the session
* Program will run as normal, reading/writing standard input/output/error
* Bash prints the PID and immediately assumes control 
* PID is accessible through `$!` special variable:

```sh
$ python -c 'import time;time.sleep(10);print("Done")' &
[1] 21919
$ echo $!
21919
$ Done
[1]+  Done                    python -c 'import time;time.sleep(10);print("Done")'
```

* Pressing `ctrl+z` suspends a running program and returns control to bash
* Variable `$!` contains the PID of most recent backgrounded process
* `fg` brings it back to the foreground and hands it control, `bg` sends it to run the background:

```sh
$ python -c 'import time;time.sleep(10);print("Done")'
^Z
[1]+  Stopped                 python -c 'import time;time.sleep(10);print("Done")'
$ bg
[1]+ python -c 'import time;time.sleep(10);print("Done")' &
$ echo $!
21919
$ Done
[1]+  Done                    python -c 'import time;time.sleep(10);print("Done")'
```

* Other useful key commands for process control:
  * `ctrl+c`: attempt to kill current command, this sends the `SIGINT` signal to the process which can be ignored
  * `ctrl+d`: exit current shell
* Other key commands for editing control available, see `man bash` (`man` is the manual program, another is `info`)
  
```sh
$ python -c 'import time;time.sleep(10);print("Done")'
^CTraceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt
$ logout
Connection to localhost closed.

```

* Important Bash/Unit concept is data pipelines where outputs from one program get fed into inputs of others
* Bash provides syntax for piping data between commands, to and from files, and from text on the command line
* `|` used between commands to pipe output from one to input of other:

```sh
$ echo "hello, lowercase" | tr "[a-z]" "[A-Z]"
HELLO, LOWERCASE
```

* `tr` performs various text translation operations

* `<` pipes data from file to command, `>` sends output to file

```sh
$ echo "hello, lowercase" | tee lower.txt
hello, lowercase
$ tr -d l < lower.txt | wc > wordcount.txt
$ cat wordcount.txt
      1       2      14
```

* `tee` is a T-junction, sending data to given file(s) and stdout
* `wc` is word count program, produces number of lines, words, and characters

* `<<` (called "here-document") used to read text from the command line or script file until a sentinel line is encountered, useful for writing out multi-line files in a command

```sh
$ cat - << _EOF_ > multiline.txt
I am a
multi-line
file
_EOF_

$ wc < multiline.txt
3  5 23
```

* In `cat -` the `-` means concatenate data from stdin rather than a file, in this case this will be the here-document